# Folium

Visualisasi Geografis

In [36]:
# pip install folium 

In [37]:
import numpy as np 
import pandas as pd 
import folium 

## Peta Dunia

In [38]:
world_map = folium.Map()
world_map

## Peta Batam

In [39]:
batam_map = folium.Map(location=[1.054507, 	104.004120], zoom_start=12)
batam_map

# location: [latitude, longitude] --> dalam betuk decimal
# latitude pada southern hemisphere pakai tanda negatif (-)
# tiles: defaultnya street map

## Peta Surabaya

Menampilkan wilayah perairan

In [40]:
# ?folium.Map

In [41]:
folium.Map(location=[-7.250445, 112.768845], zoom_start=12, tiles='Stamen Toner')

## Peta Bandung

Menampilkan elevasi

In [42]:
folium.Map(location=[-6.914744, 107.609810], zoom_start=12, tiles='Stamen Terrain')

## Marker

In [43]:
# buat peta jakarta
jakarta_map = folium.Map(location=[-6.200000, 106.816666], zoom_start=12)

# buat marker Monas
monas_pin = folium.Marker(
    location=[-6.176132, 106.822864],
    tooltip= 'Monumen Nasional',        # label ketika dihover
    popup= 'Monas',                     # label ketika diklik
    icon= folium.Icon(
        color='red', 
        icon='home'
    )
)

# buat CircleMarker Ancol
ancol_area = folium.CircleMarker(
    location=[-6.128847, 106.833053],
    tooltip= 'Ancol',
    popup= 'Area rawan banjir',
    radius= 30,
    color='blue'
)

# memasukkan marker ke dalam peta
monas_pin.add_to(jakarta_map)
ancol_area.add_to(jakarta_map)

# menampilkan jakarta map
jakarta_map 


## Choropleth

Data Source:
* Jakarta Open Data, https://data.jakarta.go.id/dataset/pelaporan-perpindahan-penduduk-keluar-provinsi-dki-jakarta-tahun-2020-berdasarkan-kelurahan/resource/134253ff-2915-44b8-a1b3-2b1869366ed0
* Jakarta by Kelurahan geo public, https://pstyd.carto.com/tables/jakarta_by_kelurahan_geo/public

In [44]:
df = pd.read_csv('pendatang_DKI_Maret_2020.csv')
df.head()

,tahun,bulan,kota_kabupaten,kecamatan,kelurahan,jenis_kelamin,jumlah
0,2020,3,ADM. KEPULAUAN SERIBU,KEPULAUAN SERIBU UTARA,PULAU PANGGANG,Laki-Laki,0.0
1,2020,3,ADM. KEPULAUAN SERIBU,KEPULAUAN SERIBU UTARA,PULAU KELAPA,Laki-Laki,2.0
2,2020,3,ADM. KEPULAUAN SERIBU,KEPULAUAN SERIBU UTARA,PULAU HARAPAN,Laki-Laki,1.0
3,2020,3,ADM. KEPULAUAN SERIBU,KEPULAUAN SERIBU SELATAN,PULAU UNTUNG JAWA,Laki-Laki,3.0
4,2020,3,ADM. KEPULAUAN SERIBU,KEPULAUAN SERIBU SELATAN,PULAU TIDUNG,Laki-Laki,3.0


In [45]:
# totalkan jumlah pendatang tiap kelurahan
df = df.groupby('kelurahan').sum()[['jumlah']].reset_index()
df 

C:\Users\user\AppData\Local\Temp\ipykernel_4292\2681890410.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('kelurahan').sum()[['jumlah']].reset_index()


,kelurahan,jumlah
0,ANCOL,17.0
1,ANGKE,22.0
2,BALEKAMBANG,30.0
3,BALI MESTER,4.0
4,BAMBU APUS,26.0
...,...,...
262,UTAN KAYU SELATAN,30.0
263,UTAN KAYU UTARA,25.0
264,UTAN PANJANG,13.0
265,WARAKAS,22.0


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   kelurahan  267 non-null    object 
 1   jumlah     267 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.3+ KB


In [47]:
# missing value diisi dengan nilai 0
df['jumlah'] = df['jumlah'].fillna(value=0)
df.isna().sum()

kelurahan    0
jumlah       0
dtype: int64

In [48]:
# buat peta jakarta
jakarta_map = folium.Map(location=[-6.200000, 106.816666], zoom_start=12)

# file geojson jakarta (untuk membuat batas kelurahan)
jakarta_geojson = r'jakarta_by_kelurahan_geo.geojson'

# Choropleth
chor_kelurahan = folium.Choropleth(
    geo_data= jakarta_geojson,          # file geojson (data geospatial)
    data= df,                           # dataframe pandas
    columns= ['kelurahan', 'jumlah'],   # kolom dataframe yg dipakai
    key_on= 'feature.properties.name'   # indexing nama kelurahan dlm file geojson untuk join dengan dataframe pandas
)

# Masukkan choropleth ke dalam peta jakarta
chor_kelurahan.add_to(jakarta_map)

jakarta_map

In [49]:
df[df['kelurahan'].str.contains('SETIA BUDI')]

# wilayah yg berwarna hitam karena datanya ada di geojson tapi tidak ada di dataframe pandas atau sebaliknya
# contoh: di geojson tertulis 'SETIABUDI' tapi di dataframe pandas tertulis 'SETIA BUDI'

,kelurahan,jumlah
230,SETIA BUDI,3.0


In [50]:
df[df['kelurahan'].str.contains('JATIPULO')]

,kelurahan,jumlah
75,JATIPULO,33.0


In [51]:
# Ubah nama kelurahan di dataframe pandas dari 'SETIA BUDI' menjadi 'SETIABUDI' agar sama dengan nama di geojson
df['kelurahan'] = df['kelurahan'].replace('SETIA BUDI', 'SETIABUDI')

In [52]:
# cek apakah sudah terubah
df[df['kelurahan'].str.contains('SETIABUDI')] 

,kelurahan,jumlah
230,SETIABUDI,3.0


In [53]:
# bins untuk legend
bins_jumlah = df['jumlah'].quantile([0, 0.25, 0.50, 0.75, 1])
bins_jumlah 

0.00      0.0
0.25     13.0
0.50     24.0
0.75     36.0
1.00    145.0
Name: jumlah, dtype: float64

In [54]:
len(df)/4

66.75

In [55]:
# Setelah ada 'SETIABUDI' 

# buat peta jakarta
jakarta_map = folium.Map(location=[-6.200000, 106.816666], zoom_start=12)

# file geojson jakarta (untuk membuat batas kelurahan)
jakarta_geojson = r'jakarta_by_kelurahan_geo.geojson'

# Choropleth
chor_kelurahan = folium.Choropleth(
    geo_data= jakarta_geojson,          # file geojson (data geospatial)
    data= df,                           # dataframe pandas
    columns= ['kelurahan', 'jumlah'],   # kolom dataframe yg dipakai
    key_on= 'feature.properties.name',  # indexing nama kelurahan dlm file geojson untuk join dengan dataframe pandas
    fill_color= 'YlOrRd',
    fill_opacity= 0.7,
    line_opacity= 0.2,
    bins= bins_jumlah                   # bins untuk legend
)

# Masukkan choropleth ke dalam peta jakarta
chor_kelurahan.add_to(jakarta_map)

jakarta_map

In [56]:
df[df['kelurahan'].str.contains('ANCOL')]

,kelurahan,jumlah
0,ANCOL,17.0


# Exercise

In [57]:
df = pd.read_csv('melb_data.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


## No.1 

- Tampilkan lokasi rumah paling mahal
- Tandai dengan icon pin warna hitam
- Beri tooltip berupa 'Rumah Termahal'
- Beri popup berupa alamat/address

In [58]:
maksimal = df['Price'].max()

In [59]:
df[df['Price'] == maksimal]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
12094,Mulgrave,35 Bevis St,3,h,9000000.0,PI,Hall,29/07/2017,18.8,3170.0,...,1.0,1.0,744.0,117.0,1960.0,Monash,-37.93168,145.16126,South-Eastern Metropolitan,7113.0


In [60]:
# buat peta jakarta
melbourne_map = folium.Map(location=[-37.840935, 144.946457])

# buat marker Monas
maksimal_pin = folium.Marker(
    location=[-37.93168, 145.16126],
    tooltip= 'Rumah Termahal',        # label ketika dihover
    popup= '35 Bevis St',                     # label ketika diklik
    icon= folium.Icon(
        color='black', 
        icon='pin'
    )
)

maksimal_pin.add_to(melbourne_map)
melbourne_map

## No.2

- Ceritanya kita sedang mencari apartemen (type= 'u') dengan 2 kamar tidur (Bedroom2) dengan harga yg murah
- Coba tampilkan peta kota melbourne
- Berikan marker pada 5 apartemen dengan harga paling murah
- Berikan tooltip berupa harga rumah
- Berikan popup berupa Suburb dan Address-nya

In [61]:
df

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [70]:
df[(df['Type'] == 'u')&(df['Bedroom2']==2)]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
10,Abbotsford,411/8 Grosvenor St,2,u,700000.0,VB,Jellis,12/11/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra,-37.81100,145.00670,Northern Metropolitan,4019.0
12,Abbotsford,123/56 Nicholson St,2,u,750000.0,S,Biggin,12/11/2016,2.5,3067.0,...,2.0,1.0,0.0,94.0,2009.0,Yarra,-37.80780,144.99650,Northern Metropolitan,4019.0
23,Abbotsford,6/219 Nicholson St,2,u,500000.0,S,Collins,18/06/2016,2.5,3067.0,...,1.0,1.0,0.0,60.0,1970.0,Yarra,-37.80150,144.99720,Northern Metropolitan,4019.0
44,Airport West,2/49 McIntosh St,2,u,500000.0,VB,Barry,4/03/2017,13.5,3042.0,...,1.0,1.0,168.0,86.0,2011.0,Moonee Valley,-37.72150,144.88640,Western Metropolitan,3464.0
49,Airport West,3/19 Hart St,2,u,515000.0,S,Barry,7/11/2016,13.5,3042.0,...,1.0,1.0,141.0,73.0,2013.0,Moonee Valley,-37.72980,144.88830,Western Metropolitan,3464.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,Hoppers Crossing,3/14 Moffatt Cr,2,u,315000.0,S,Triwest,26/08/2017,18.4,3029.0,...,1.0,1.0,150.0,NaN,NaN,NaN,-37.87715,144.69863,Western Metropolitan,13830.0
13482,Malvern East,2002 Malvern Rd,2,u,651000.0,SP,Jellis,26/08/2017,8.4,3145.0,...,1.0,1.0,129.0,97.0,1940.0,NaN,-37.87798,145.06731,Southern Metropolitan,8801.0
13495,Moonee Ponds,1/53 Buckley St,2,u,435000.0,S,Nelson,26/08/2017,6.2,3039.0,...,1.0,1.0,1475.0,66.0,1970.0,NaN,-37.75799,144.92354,Western Metropolitan,6232.0
13510,Nunawading,3/39 Lemon Gr,2,u,710000.0,S,Jellis,26/08/2017,15.4,3131.0,...,1.0,1.0,903.0,NaN,1985.0,NaN,-37.80640,145.18452,Eastern Metropolitan,4973.0


In [63]:
harga_rata = df[(df['Type'] == 'u')&(df['Bedroom2']==2)]['Price'].mean()

In [64]:
df_murah = df[(df['Type'] == 'u')&(df['Price'] < harga_rata)&(df['Bedroom2']==2)]
df_murah

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
23,Abbotsford,6/219 Nicholson St,2,u,500000.0,S,Collins,18/06/2016,2.5,3067.0,...,1.0,1.0,0.0,60.0,1970.0,Yarra,-37.80150,144.99720,Northern Metropolitan,4019.0
44,Airport West,2/49 McIntosh St,2,u,500000.0,VB,Barry,4/03/2017,13.5,3042.0,...,1.0,1.0,168.0,86.0,2011.0,Moonee Valley,-37.72150,144.88640,Western Metropolitan,3464.0
49,Airport West,3/19 Hart St,2,u,515000.0,S,Barry,7/11/2016,13.5,3042.0,...,1.0,1.0,141.0,73.0,2013.0,Moonee Valley,-37.72980,144.88830,Western Metropolitan,3464.0
50,Airport West,2/62 Clydesdale Rd,2,u,530000.0,S,Nelson,10/09/2016,13.5,3042.0,...,1.0,1.0,120.0,94.0,2012.0,Moonee Valley,-37.72340,144.87920,Western Metropolitan,3464.0
54,Airport West,145a Victory Rd,2,u,440000.0,S,Barry,13/08/2016,13.5,3042.0,...,1.0,1.0,192.0,NaN,NaN,Moonee Valley,-37.71770,144.87870,Western Metropolitan,3464.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13129,Brunswick East,2/162 Lygon St,2,u,522000.0,SP,Jellis,23/09/2017,4.0,3057.0,...,1.0,1.0,186.0,NaN,NaN,NaN,-37.77315,144.97161,Northern Metropolitan,5533.0
13179,Flemington,2/121 Wellington St,2,u,452500.0,S,Nelson,23/09/2017,3.4,3031.0,...,1.0,1.0,807.0,54.0,1970.0,NaN,-37.78300,144.93204,Northern Metropolitan,3593.0
13439,Greensborough,2/5 McDowell St,2,u,540000.0,S,Darren,26/08/2017,16.1,3088.0,...,1.0,1.0,208.0,NaN,1995.0,NaN,-37.70791,145.09575,Northern Metropolitan,8524.0
13453,Hoppers Crossing,3/14 Moffatt Cr,2,u,315000.0,S,Triwest,26/08/2017,18.4,3029.0,...,1.0,1.0,150.0,NaN,NaN,NaN,-37.87715,144.69863,Western Metropolitan,13830.0


In [65]:
melbourne_map

In [66]:
df_murah

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
23,Abbotsford,6/219 Nicholson St,2,u,500000.0,S,Collins,18/06/2016,2.5,3067.0,...,1.0,1.0,0.0,60.0,1970.0,Yarra,-37.80150,144.99720,Northern Metropolitan,4019.0
44,Airport West,2/49 McIntosh St,2,u,500000.0,VB,Barry,4/03/2017,13.5,3042.0,...,1.0,1.0,168.0,86.0,2011.0,Moonee Valley,-37.72150,144.88640,Western Metropolitan,3464.0
49,Airport West,3/19 Hart St,2,u,515000.0,S,Barry,7/11/2016,13.5,3042.0,...,1.0,1.0,141.0,73.0,2013.0,Moonee Valley,-37.72980,144.88830,Western Metropolitan,3464.0
50,Airport West,2/62 Clydesdale Rd,2,u,530000.0,S,Nelson,10/09/2016,13.5,3042.0,...,1.0,1.0,120.0,94.0,2012.0,Moonee Valley,-37.72340,144.87920,Western Metropolitan,3464.0
54,Airport West,145a Victory Rd,2,u,440000.0,S,Barry,13/08/2016,13.5,3042.0,...,1.0,1.0,192.0,NaN,NaN,Moonee Valley,-37.71770,144.87870,Western Metropolitan,3464.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13129,Brunswick East,2/162 Lygon St,2,u,522000.0,SP,Jellis,23/09/2017,4.0,3057.0,...,1.0,1.0,186.0,NaN,NaN,NaN,-37.77315,144.97161,Northern Metropolitan,5533.0
13179,Flemington,2/121 Wellington St,2,u,452500.0,S,Nelson,23/09/2017,3.4,3031.0,...,1.0,1.0,807.0,54.0,1970.0,NaN,-37.78300,144.93204,Northern Metropolitan,3593.0
13439,Greensborough,2/5 McDowell St,2,u,540000.0,S,Darren,26/08/2017,16.1,3088.0,...,1.0,1.0,208.0,NaN,1995.0,NaN,-37.70791,145.09575,Northern Metropolitan,8524.0
13453,Hoppers Crossing,3/14 Moffatt Cr,2,u,315000.0,S,Triwest,26/08/2017,18.4,3029.0,...,1.0,1.0,150.0,NaN,NaN,NaN,-37.87715,144.69863,Western Metropolitan,13830.0


In [67]:
lima_murah = df_murah.sort_values(by='Price').head(5).reset_index().drop(columns='index')
lima_murah

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Albion,8/6 Ridley St,1,u,145000.0,PI,Biggin,28/05/2016,13.9,3020.0,...,1.0,1.0,36.0,NaN,NaN,Brimbank,-37.7833,144.8266,Western Metropolitan,2185.0
1,Albion,5/25 Ridley St,2,u,190000.0,SP,Burnham,30/07/2016,13.9,3020.0,...,1.0,1.0,0.0,NaN,NaN,Brimbank,-37.7839,144.8239,Western Metropolitan,2185.0
2,Reservoir,1/164 Leamington St,1,u,216000.0,S,Brad,24/09/2016,11.2,3073.0,...,1.0,1.0,0.0,45.0,1970.0,Darebin,-37.7111,144.9919,Northern Metropolitan,21650.0
3,Glenroy,14/854 Pascoe Vale Rd,2,u,230000.0,PI,Raine,18/03/2017,13.0,3046.0,...,1.0,1.0,1996.0,NaN,NaN,Moreland,-37.7020,144.9154,Northern Metropolitan,8870.0
4,Reservoir,4/1067 High St,2,u,240000.0,PI,Stockdale,26/07/2016,11.2,3073.0,...,1.0,1.0,82.0,NaN,1970.0,Darebin,-37.6983,145.0104,Northern Metropolitan,21650.0


In [68]:
a=1
for i in range(len(lima_murah)):
    for i in range(a):
        b = [lima_murah['Lattitude'][i],lima_murah['Longtitude'][i]]
        a <= 5
b

[-37.7833, 144.8266]

In [69]:
# buat peta jakarta
melbourne_map = folium.Map(location=[-37.840935, 144.946457])


for i in range (len(lima_murah)):
    murah_pin = folium.Marker(
        location= [lima_murah['Lattitude'][i],lima_murah['Longtitude'][i]],
        tooltip= int(lima_murah['Price'][i]),                                         # label ketika dihover
        popup= df['Suburb'][i] + ' ' + df['Address'][i],                        # label ketika diklik
        icon= folium.Icon( 
            color='green',
            icon='usd'
        )
    )
    murah_pin.add_to(melbourne_map)
        
maksimal_pin.add_to(melbourne_map)
melbourne_map